# 🔍 Test Pancake POS API - Kiểm tra giới hạn page_size 

## 📋 Mục tiêu của notebook này

**Mục tiêu chính**: Tìm ra giá trị `page_size` tối đa mà Pancake POS API có thể xử lý để tối ưu hóa quá trình ETL (Extract, Transform, Load).

## 🤔 Tại sao cần kiểm tra page_size?

### 1. **Xác định giới hạn API**
- Tài liệu API có thể không ghi rõ `page_size` tối đa
- Nếu gửi request quá lớn, API sẽ trả về lỗi HTTP (400/422/500)
- Cần biết chính xác ngưỡng để tránh lỗi trong quá trình ETL

### 2. **Tối ưu hiệu suất ETL**
- `page_size` lớn → ít request hơn → tải dữ liệu nhanh hơn
- `page_size` quá lớn → dễ timeout hoặc nghẽn mạng
- Cần tìm "điểm cân bằng" giữa tốc độ và độ ổn định  

### 3. **Tránh vượt rate limit**
- API thường giới hạn số request/giây hoặc request/phút
- `page_size` nhỏ → nhiều request → dễ vượt giới hạn
- `page_size` hợp lý → ít request hơn → tiết kiệm quota

### 4. **Đảm bảo tính toàn vẹn dữ liệu**  
   - Nếu không kiểm tra, có thể mất dữ liệu (API tự cắt kết quả).  
   - Ví dụ: yêu cầu `page_size=2000` nhưng API chỉ trả tối đa `1000`.  

**Kết luận**: Việc test `page_size` giúp tìm ra “ngưỡng vàng” để vừa giảm số lần gọi API, vừa đảm bảo dữ liệu đầy đủ, không bị lỗi.



---
## 🛠️ Bước 1: Import thư viện & Load biến môi trường

### 📚 Các thư viện cần thiết:
- **`requests`**: Để gọi API Pancake POS
- **`dotenv`**: Để đọc biến môi trường từ file `.env`
- **`os`**: Để truy cập biến môi trường hệ thống

### 🔑 Thông tin cần thiết:
- **`API_KEY`**: Khóa API từ Pancake POS
- **`SHOP_ID`**: ID của shop cần test  

In [1]:
import requests
from dotenv import load_dotenv
import os

# Load biến môi trường từ file .env
load_dotenv()
API_KEY = os.getenv("API_KEY")
SHOP_ID = os.getenv("SHOP_ID")


## 🔧 Bước 2: Định nghĩa hàm test_page_size

### 📝 Chức năng của hàm:
- **Input**: `shop_id` (ID shop) và `page_size` (số bản ghi muốn lấy)
- **Process**: Gọi API `/orders` với `page=1` và `page_size` tùy chọn
- **Output**: Hiển thị kết quả test

### 📊 Thông tin hiển thị:
- ✅ **Status code** của API response
- ✅ **Số lượng bản ghi** lấy được (nếu thành công)
- ❌ **Thông báo lỗi** (nếu thất bại)

### 🎯 Mục đích:
Kiểm tra xem API có chấp nhận giá trị `page_size` này không  


In [2]:
def test_page_size(shop_id, page_size):
    url = f"https://pos.pages.fm/api/v1/shops/{shop_id}/orders"
    params = {"api_key": API_KEY, "page": 1, "page_size": page_size}
    resp = requests.get(url, params=params)
    print(f"Page size {page_size} → status {resp.status_code}")
    
    if resp.status_code == 200:
        data = resp.json().get("data", [])
        print(f"👉 Lấy được {len(data)} bản ghi")
    else:
        print("❌ Lỗi:", resp.text[:200])

## 🧪 Bước 3: Thử nghiệm với nhiều giá trị page_size

### 🔬 Chiến lược test:
- **Các giá trị test**: 50, 100, 200, 500, 1000, 2000
- **Mục tiêu**: Tìm ra giá trị `page_size` tối đa mà API chấp nhận
- **Phương pháp**: Test từ nhỏ đến lớn để tìm ngưỡng

### 📈 Kết quả mong đợi:
- ✅ **Thành công**: API trả về status 200 với đúng số bản ghi
- ❌ **Thất bại**: API trả về status 422/400 với thông báo lỗi
- 🎯 **Mục tiêu**: Xác định được `page_size` tối đa  


In [4]:
for ps in [50, 100, 200, 500, 1000, 2000]:
    test_page_size(SHOP_ID, ps)

Page size 50 → status 200
👉 Lấy được 50 bản ghi
Page size 100 → status 200
👉 Lấy được 100 bản ghi
Page size 200 → status 200
👉 Lấy được 200 bản ghi
Page size 500 → status 200
👉 Lấy được 500 bản ghi
Page size 1000 → status 200
👉 Lấy được 1000 bản ghi
Page size 2000 → status 422
❌ Lỗi: {"error":"page size must be less than or equal to 1000"}


## 🎯 Kết luận

### ✅ **Kết quả test:**
Pancake POS API cho phép tối đa **1000 bản ghi mỗi lần gọi** (`page_size ≤ 1000`)

### 🚀 **Khuyến nghị cho ETL:**
- **Đặt `page_size = 1000`** để giảm số lần request và tối ưu tốc độ tải dữ liệu
- **Vẫn cần phân trang** (page=1,2,3,...) cho dataset lớn
- **Mỗi trang có thể lấy tối đa 1000 bản ghi**

### 📊 **Lợi ích:**
- ⚡ **Tăng tốc độ**: Ít request hơn → tải dữ liệu nhanh hơn
- 💰 **Tiết kiệm quota**: Giảm số lần gọi API
- 🛡️ **Tránh lỗi**: Không vượt quá giới hạn API
- 📈 **Hiệu quả**: Tối ưu hóa quá trình ETL  
